In [1]:
import os
import shutil
import json
import zlib
import traceback
import urllib.request

import pandas as pd
from Bio import Entrez
from Bio import SearchIO
from Bio.SearchIO.HmmerIO.hmmer3_text import Hmmer3TextParser

from eaglib.alignment import SeqsProfileInfo, SeqsProfile
from eaglib.seqs import SeqsDict, load_fasta_to_dict

In [7]:
! pip uninstall -y EAGLE
! pip install ../../dist/EAGLE-0.0.1-py3-none-any.whl

Found existing installation: EAGLE 0.0.1
Uninstalling EAGLE-0.0.1:
  Successfully uninstalled EAGLE-0.0.1
Processing /media/olga/Data/Denis/EAGLE/dist/EAGLE-0.0.1-py3-none-any.whl


In [2]:
def get_taxonomy(tax_id):
    tax_keys = ["superkingdom", "phylum", "clade", "class", "order", "family", "genus", "species"]
    tax_dict = {tax_key: None for tax_key in tax_keys}
    
    record = Entrez.efetch(db="taxonomy", id=tax_id, retmode='xml')
    tax_info = Entrez.read(record)[0]
    tax_dict["species"] = tax_info['ScientificName']
    
    for lin_tax in tax_info['LineageEx']:
        if lin_tax['Rank'] in tax_dict:
            tax_dict[lin_tax['Rank']] = lin_tax['ScientificName']
            
    return [tax_dict[tax_key] for tax_key in tax_keys]

In [3]:
arch_16s_profile = SeqsProfile(SeqsProfileInfo.load_from_dict({
    'name': '16S_rRNA_archaea',
    'path': '16S_rRNA_archaea.cm',
    'type': 'rna',
    'weight': 1.0,
    'method': 'infernal'
}))

In [4]:
hsp70_profile = SeqsProfile(SeqsProfileInfo.load_from_dict({
    'name': 'HSP70',
    'path': 'HSP70.hmm',
    'type': 'protein',
    'weight': 1.0,
    'method': 'hmmer'
}))

In [5]:
assembly_summary_path = "archaea_assembly_summary.txt"
Entrez.email = "moshenskydenis@gmail.com"
db_dir = "archaea"

processed_ac = list()
arch_df = pd.read_csv(assembly_summary_path, sep="\t")
# arch_df.query("assembly_level=='Complete Genome' & refseq_category!='representative genome'")
for _, row in arch_df.query("assembly_level=='Complete Genome'").iterrows():
    ac = row['assembly_accession']  # id field in genomes_table
    asm = row['asm_name']
    taxonomy = get_taxonomy(row['species_taxid'])
    name = row['organism_name'] + ("" if pd.isna(row['infraspecific_name']) else " " + row['infraspecific_name'])
    ftp_prefix = (row['ftp_path'] + "/" + ac + "_" + asm).replace(" ", "_")
    fna_seq = [ftp_prefix+"_genomic.fna.gz"]
    # fna_seq = [os.path.join(db_dir, ac+"_genomic.fna")] #
    btc_seqs = [os.path.join(db_dir, ac+"_btc.fasta")]
    rna_path = os.path.join(db_dir, ac+"_rna_from_genomic.fna")
    tcds_path = os.path.join(db_dir, ac+"_translated_cds.faa")
    btc_seqs_dict = dict()
    
    try:
    #     with open(fna_seq[0], 'wb') as gfna_f: #
    #         gfna_f.write(zlib.decompress(urllib.request.urlopen(ftp_prefix+"_genomic.fna.gz").read(), 15+32)) #
        with open(rna_path, 'wb') as rna_f:
            rna_f.write(zlib.decompress(urllib.request.urlopen(ftp_prefix+"_rna_from_genomic.fna.gz").read(), 15+32))
        psr_rna_df = arch_16s_profile.search(seqdb=rna_path, threads=4)
        if not psr_rna_df.empty:
            max_score_rna = psr_rna_df.loc[psr_rna_df["score"].idxmax()]
            btc_seqs_dict[arch_16s_profile.name] = load_fasta_to_dict(fasta_path=rna_path)[max_score_rna["target name"]][max_score_rna["seq from"]-1: max_score_rna["seq to"]]                     
        
        with open(tcds_path, 'wb') as tcds_f:
            tcds_f.write(zlib.decompress(urllib.request.urlopen(ftp_prefix+"_translated_cds.faa.gz").read(), 15+32))
        psr_hsp_df = hsp70_profile.search(seqdb=tcds_path, threads=4)
        if not psr_hsp_df.empty:
            max_score_hsp = psr_hsp_df.loc[psr_hsp_df["domain_score"].idxmax()]
            btc_seqs_dict[hsp70_profile.name] = load_fasta_to_dict(fasta_path=tcds_path)[max_score_hsp["target name"]][max_score_hsp["ali_from"]-1: max_score_hsp["ali_to"]]                     
        
        SeqsDict.load_from_dict(btc_seqs_dict).dump(btc_seqs[0])
        processed_ac.append({"id": ac, "name": name, "taxonomy": taxonomy, "btc_seqs": btc_seqs, "fna_seq": fna_seq})
    except:
        print(traceback.format_exc())
    print(ac, name)

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_000762265.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_anXXhVH5xC.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                   start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ------------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1293.4  12.7  lcl|NZ_CP006933.1_rrna_4       3   1475 + hmm     - 0.56  [locus_tag=BRM9_RS01355] [db_xref=RFAM:RF01

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  HSP70.hmm
# target sequence database:        archaea/GCF_000762265.1_translated_cds.faa
# per-dom hits tabular output:     HSP70_out_wyPnMhHncY.psr
# number of worker threads:        4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       HSP70  [M=599]
Accession:   PF00012.23
Description: Hsp70 protein
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                                   Description
    ------- ------ -----    ------- ------ -----   ---- --  --------                                   

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_001458655.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_JNhaF6CTeZ.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                   start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ------------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1293.4  12.7  lcl|NZ_LN734822.1_rrna_30      3   1475 + hmm     - 0.56  [locus_tag=MB9_RS08735] [db_xref=RFAM:RF019

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  HSP70.hmm
# target sequence database:        archaea/GCF_001458655.1_translated_cds.faa
# per-dom hits tabular output:     HSP70_out_LNgdKscHSv.psr
# number of worker threads:        4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       HSP70  [M=599]
Accession:   PF00012.23
Description: Hsp70 protein
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                                   Description
    ------- ------ -----    ------- ------ -----   ---- --  --------                                   

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_002813085.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_zKxTsQ9QpQ.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                   start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ------------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1275.5   6.2  lcl|NZ_CP017803.1_rrna_12      2   1475 + hmm     - 0.53  [locus_tag=BK798_RS02330] [db_xref=RFAM:RF0

...

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_000145295.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_gDrbYyzuoq.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                 start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ----------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1286.6  16.8  lcl|NC_014408.1_rrna_8       3   1481 + hmm     - 0.58  [locus_tag=MTBMA_RS01620] [db_xref=RFAM:RF01959] 

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  HSP70.hmm
# target sequence database:        archaea/GCF_000145295.1_translated_cds.faa
# per-dom hits tabular output:     HSP70_out_17EHaDNT9s.psr
# number of worker threads:        4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       HSP70  [M=599]
Accession:   PF00012.23
Description: Hsp70 protein
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                                 Description
    ------- ------ -----    ------- ------ -----   ---- --  --------                                 ----

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_900036045.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_kOW8fcABHY.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                   start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ------------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1261.4  13.7  lcl|NZ_LT158599.1_rrna_12      2   1466 + hmm     - 0.56  [locus_tag=MAB1_RS02835] [db_xref=RFAM:RF01

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  HSP70.hmm
# target sequence database:        archaea/GCF_900036045.1_translated_cds.faa
# per-dom hits tabular output:     HSP70_out_A3UgH4i5ft.psr
# number of worker threads:        4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       HSP70  [M=599]
Accession:   PF00012.23
Description: Hsp70 protein
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                                   Description
    ------- ------ -----    ------- ------ -----   ---- --  --------                                   

KeyboardInterrupt: 

In [5]:
arch_16s_profile_ss = SeqsProfile.build(mult_aln="RF01959.stockholm", name="16S_rRNA_archaea_ss", method="infernal", seqs_type="rna", noss=False)

# cmbuild :: covariance model construction from multiple sequence alignments
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# CM file:                                            16S_rRNA_archaea_ss.cm
# alignment file:                                     RF01959.stockholm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#                                                                      rel entropy
#                                                                      -----------
# idx    name                     nseq eff_nseq   alen  clen  bps bifs    CM   HMM description
# ------ -------------------- -------- -------- ------ ----- ---- ---- ----- ----- -----------
       1 SSU_rRNA_archaea           86     1.06   1958  1478  457   30 0.589 0.302 Archaeal small subunit ribosomal RNA
#
# CPU time

Killed


In [5]:
psr = arch_16s_profile.search(seqdb="archaea/GCF_000762265.1_rna_from_genomic.fna", threads=4)

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.4 (Dec 2020)
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         16S_rRNA_archaea.cm
# target sequence database:              archaea/GCF_000762265.1_rna_from_genomic.fna
# tabular output of hits:                16S_rRNA_archaea_out_4KCpOJatpl.psr
# number of worker threads:              4 [--cpu]
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       RF01959  [CLEN=1478]
Hit scores:
 rank     E-value  score  bias  sequence                   start    end   mdl trunc   gc  description
 ----   --------- ------ -----  ------------------------- ------ ------   --- ----- ----  -----------
  (1) !         0 1293.4  12.7  lcl|NZ_CP006933.1_rrna_4       3   1475 + hmm     - 0.56  [locus_tag=BRM9_RS01355] [db_xref=RFAM:RF01

In [6]:
psr["score"].idxmax()

0

In [7]:
psr1 = hsp70_profile.search(seqdb="archaea/GCF_000762265.1_translated_cds.faa", threads=4)

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  HSP70.hmm
# target sequence database:        archaea/GCF_000762265.1_translated_cds.faa
# per-dom hits tabular output:     HSP70_out_GFNDWEhvCq.psr
# number of worker threads:        4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       HSP70  [M=599]
Accession:   PF00012.23
Description: Hsp70 protein
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                                   Description
    ------- ------ -----    ------- ------ -----   ---- --  --------                                   

In [12]:
psr1[['target name', 'seq_score', 'domain_score', 'ali_from', 'ali_to']]

,target name,seq_score,domain_score,ali_from,ali_to
0,lcl|NZ_CP006933.1_prot_WP_048085785.1_2239,760.4,73.0,7,85
1,lcl|NZ_CP006933.1_prot_WP_048085785.1_2239,760.4,690.1,85,584
2,lcl|NZ_CP006933.1_prot_WP_048085701.1_2093,50.6,4.4,22,70
3,lcl|NZ_CP006933.1_prot_WP_048085701.1_2093,50.6,45.3,112,342


In [8]:
df = pd.DataFrame(columns=['target name', 'seq_score', 'domain_score', 'ali_from', 'ali_to'])
df['seq_score'].idxmax()

TypeError: reduction operation 'argmax' not allowed for this dtype